In [1]:
pip install -U imbalanced-learn

     |████████████████████████████████| 174kB 2.7MB/s 
     |████████████████████████████████| 6.8MB 9.0MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: imbalanced-learn 0.4.3
    Uninstalling imbalanced-learn-0.4.3:
      Successfully uninstalled imbalanced-learn-0.4.3


## Import of the libraries

In [6]:
# Import of all the required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats
import seaborn as sns
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, f1_score
from sklearn.model_selection import train_test_split
import imblearn

## Import of the Dataset

The data will be imported from a pickle file saved after the data analysis notebook

In [4]:
# Connecting the Google drive with Colab

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Import of the train and test datasets into the Panda dataframe from Google drive 
# To load data from the local drive the location below can be changed accordingly


train_df = pickle.load(open("/content/drive/My Drive/Colab Notebooks/train_df_final.pkl", "rb"))
test_df = pickle.load(open("/content/drive/My Drive/Colab Notebooks/test_df_final.pkl", "rb"))

train_df
test_df

,V1,V4,V5,V7,V8,V9,V10,V13,V14,V16,V20,V21,V22,V24,V27,V28,Amount,Class
Id,,,,,,,,,,,,,,,,,,
1,-0.584007,0.001509,0.292260,1.120755,-0.166826,-0.501695,-0.694423,-0.276469,0.284019,0.265888,-0.106804,0.167209,0.361133,0.375231,0.102098,0.144502,42.81,0
2,1.326714,-0.979830,-0.625116,-0.636415,0.050208,-0.785860,0.815473,-1.025594,0.271240,1.751346,0.164916,0.109578,-0.011447,-0.859332,-0.021529,0.003230,75.00,0
3,2.097691,-0.011746,0.597501,0.130716,-0.174237,0.383371,0.171848,0.344720,0.589561,0.230686,-0.192933,0.233496,0.825427,-1.115397,-0.020643,-0.079921,2.29,0
4,-1.593912,-0.236255,-0.002224,-0.392776,-2.005520,-0.306309,-0.253465,1.539975,-0.407657,0.687552,-0.258684,1.995801,-1.554302,-0.531345,0.346683,0.104860,8.99,0
5,1.863194,3.853830,0.220877,-0.296455,0.207090,-0.487512,1.454433,-0.153871,-0.147691,0.606259,-0.311545,0.118870,0.611417,-0.387779,-0.003506,-0.061238,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85439,-0.753070,-1.448546,1.130943,1.167232,0.201294,-0.245013,0.113600,-1.027914,1.212622,-0.053734,0.050720,0.202703,0.725051,-1.016408,0.469657,0.304103,0.75,0
85440,-0.539994,0.514863,-0.043655,0.851781,-0.059080,0.319422,-0.573884,-1.541622,-0.192496,-0.702496,-0.173193,0.059047,0.441880,0.574042,-0.007875,-0.067811,40.00,0
85441,-1.833594,-1.464593,-0.448623,-0.496368,0.481275,-1.151529,-0.210526,0.895314,-0.274270,1.832300,0.754963,0.363656,0.279139,-0.118669,0.209083,-0.049870,140.00,0


## Unbalance Dataset Approaches

Reference:
1. https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/#:~:text=Imbalanced%20data%20typically%20refers%20to,with%20100%20instances%20(rows).

2. https://www.kaggle.com/janiobachmann/credit-fraud-dealing-with-imbalanced-datasets/notebook


Below there are the approaches



### 1. Model with Different Algorithms
Create the model with two different algorithms to see the performance with the data as it is

  Will evaluate the models with MSE and F1 score

  The model tested will be random forrest and SVC

In [8]:
# Dividing the training set in training and validation

train_set, validation_set = train_test_split(train_df, test_size=0.2)

validation_set

,V1,V4,V5,V7,V8,V9,V10,V13,V14,V16,V20,V21,V22,V24,V27,V28,Amount,Class
Id,,,,,,,,,,,,,,,,,,
23026,-0.497896,-1.468950,-1.430410,-1.053097,0.570986,0.398328,-0.352973,1.710445,1.255569,2.175176,-0.058218,0.589994,1.630531,0.299048,0.106233,0.157483,44.90,0
188987,2.045788,-1.174332,-0.148105,-0.370538,-0.212934,-0.761823,0.955472,1.068328,-0.013726,1.430400,0.259670,0.408209,0.968361,-1.088994,-0.046802,-0.063225,114.20,0
38906,0.080669,0.758409,0.705058,1.243962,-0.497103,-0.282741,0.704912,0.788760,0.038618,-0.593659,0.304160,0.036950,0.612540,0.058848,0.387444,0.064717,0.74,0
81930,2.052427,0.277935,0.337999,0.334629,-0.227039,0.267655,0.293539,-0.989043,0.761661,-0.296433,-0.269148,-0.006518,0.121300,-0.356498,-0.103929,-0.091958,8.73,0
164212,0.093641,-0.526791,1.131571,2.251729,-0.682577,-0.776130,-0.704394,0.459855,0.630607,-1.020189,0.001119,0.424370,1.265556,0.061315,-0.027426,0.017366,90.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79352,1.946018,-0.173366,-1.786292,-1.483279,0.103143,0.512129,0.761682,0.581414,-0.615531,-0.729622,-0.465529,-0.113152,0.210059,-0.080033,0.030633,-0.018940,68.00,0
117708,-0.299345,-1.719362,0.370424,0.650405,-0.141236,-1.842230,0.257093,0.706017,0.244492,-1.223635,-0.387776,0.000864,0.425196,-0.421312,0.064067,0.122865,33.00,0
110765,1.725363,0.475997,0.123668,0.261293,-0.162334,0.548930,-0.342127,-1.426081,-0.228198,0.523327,0.036599,0.310039,0.618259,-0.490074,-0.042271,-0.020992,159.90,0


In [71]:
# definition of the function to model and score the classification with the random forest
def random_forest_classifier(test, validation):
  x_test = test.loc[:, test.columns != 'Class']
  y_test = test["Class"]
  x_val = validation.loc[:, validation.columns != 'Class']
  y_val = validation["Class"]
  model= RandomForestClassifier(n_estimators=500, max_depth=400, max_samples=80000)
  model.fit(x_test, y_test)

  test_prediction = model.predict(x_test)
  print("training set score ", model.score(x_test, y_test))
  f1_score_test = f1_score(y_test, test_prediction)
  print("F1 training set score ", f1_score_test)
  mse_test = mean_squared_error(y_test, test_prediction)
  print("MSE training set score ", mse_test)


  validation_prediction = model.predict(x_val)
  print("validation set score ", model.score(x_val, y_val))
  f1_score_validation = f1_score(y_val, validation_prediction)
  print("F1 training set score ", f1_score_validation)
  mse_validation = mean_squared_error(y_val, validation_prediction)
  print("MSE training set score ", mse_validation)
  return model

In [72]:
model_1 = random_forest_classifier(train_set, validation_set)

training set score  0.999691800639042
F1 training set score  0.9073724007561436
MSE training set score  0.0003081993609580597
validation set score  0.9994213545335614
F1 training set score  0.8067226890756303
MSE training set score  0.0005786454664385629


In [75]:
# definition of the function to model and score the classification with the support vector machine
def svm(test, validation):
  x_test = test.loc[:, test.columns != 'Class']
  y_test = test["Class"]
  x_val = validation.loc[:, validation.columns != 'Class']
  y_val = validation["Class"]
  model= SVC(C=100000, kernel='rbf')
  model.fit(x_test, y_test)
  test_prediction = model.predict(x_test)
  print("training set score ", model.score(x_test, y_test))
  f1_score_test = f1_score(y_test, test_prediction)
  print("F1 training set score ", f1_score_test)
  mse_test = mean_squared_error(y_test, test_prediction)
  print("MSE training set score ", mse_test)


  validation_prediction = model.predict(x_val)
  print("validation set score ", model.score(x_val, y_val))
  f1_score_validation = f1_score(y_val, validation_prediction)
  print("F1 training set score ", f1_score_validation)
  mse_validation = mean_squared_error(y_val, validation_prediction)
  print("MSE training set score ", mse_validation)
  return model

In [76]:
model_2 = svm(train_set, validation_set)

training set score  0.9995974538958915
F1 training set score  0.8801498127340824
MSE training set score  0.0004025461041084862
validation set score  0.9993961960350206
F1 training set score  0.793103448275862
MSE training set score  0.00060380396497937


The above models have got F1 validation score of about 80% with validation set

The hyper parameters for both models have been manually tweaked to have best result as shown above

###2. Oversampling of the Dataset 

Using the Imbalance learn library to use various method to oversample

Reference:

https://github.com/scikit-learn-contrib/imbalanced-learn

https://imbalanced-learn.org/stable/index.html




In [83]:
# Oversampling of the dataset randomly 

def oversample(test):
  x_test = test.loc[:, test.columns != 'Class']
  y_test = test["Class"]
  oversampler = imblearn.over_sampling.RandomOverSampler(random_state=0)
  new_x_test, new_y_test = oversampler.fit_resample(x_test, y_test)
  df = new_x_test
  df["Class"] = new_y_test
  return df

oversample_train_set = oversample(train_set)


In [85]:
model_3 = random_forest_classifier(oversample_train_set, validation_set)

training set score  0.9999495910574536
F1 training set score  0.999949593598387
MSE training set score  5.040894254640773e-05
validation set score  0.9995219885277247
F1 training set score  0.8455284552845528
MSE training set score  0.0004780114722753346


In [ ]:
model_4 = svm(oversample_train_set, validation_set)